In [1]:
#Importing Selenium
from selenium import webdriver 
import selenium 
from selenium.webdriver.firefox.options import Options
import time 

options = Options()
options.headless = True

#Setting up webdriver client
browser = webdriver.Firefox(options = options)
browser.get("https://www.myodds.bet/")    
browser.maximize_window() #May not need to maximize thought it might help with finding elements.

In [2]:
#Finding each Match
search_box = browser.find_elements_by_css_selector('div.match-detail') 
#Closing a popup that upstructs element clicking later.
close_button = browser.find_elements_by_css_selector('.mat-raised-button')
close_button[0].click()

In [3]:
#This list is just a place holder for a database. The match scrapped below will be put into a Mongo DB.
import pandas as pd
betting_df = pd.DataFrame(columns = ['game', 'time', 'team_one', 'team_two', 'map', 'event', 'team_one_odd', 'team_two_odd'])

ticker = 0
#Iterating through each match
for box in search_box:
    game = box.text.splitlines()
    #Checking if the match have 7 text elements. This is used to filter out the matches without betting lines.
    #May have to set to >= 7, because the league or tournament may be included on this line.
    if len(game) == 7:
        #Extracting the images in the match and finding the one with a title.
        #This title is the type of game being played. (CSGO, DOTA, etc.)
        img = box.find_elements_by_tag_name('img')
        for element in img:
            game_title = element.get_attribute('title')
            if game_title != '':
                vd = element.get_attribute('title')

        #Clicking on the match to show all of the lines availbe. Named drips.
        box.click()   
        drips = box.find_elements_by_css_selector('div.detail > div:nth-child(n)')
 
        for i in range(len(drips)):
            if i == 0: #The first item corresponds to lines that relate to the whole match not an individual map.
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for line in lines:
                    if len(line.text) >= 1: #Making sure the line has text. Some don't.
                        split_line = line.text.splitlines()
                        betting_df = betting_df.append(pd.Series([vd, game[0], game[2], game[6], 'match', split_line[0], float(split_line[1]), float(split_line[2])], index = betting_df.columns), ignore_index = True)



            else:
                #Finding the map or event type.
                event_type = drips[i].text.splitlines()[0].lower().replace(' ','_').replace('.', '_')
                #The f string {i + 1} has the +1 becuase the count starts at 1 not 0 like in python
                lines = drips[i].find_elements_by_css_selector(f'div.detail > div:nth-child({i + 1}) > div:nth-child(n)')
                for k in range(len(lines)):
                    if k != 0: #The first line corresponds to the map or event type. Redundant and easier to avoid in this loop
                        if len(lines[k].text) >= 1: #Making sure the line has text. Some don't.
                            split_line = lines[k].text.splitlines()
                            betting_df = betting_df.append(pd.Series([vd, game[0], game[2], game[6], event_type, split_line[0], float(split_line[1]), float(split_line[2])], index = betting_df.columns), ignore_index = True)
    
                            
    

#         #This is here to simulate adding to database
#         dic_id = test_col.insert_one(dic).inserted_id
    ticker += 1
    print(ticker / len(search_box))

0.009900990099009901
0.019801980198019802
0.0297029702970297
0.039603960396039604
0.04950495049504951
0.0594059405940594
0.06930693069306931
0.07920792079207921
0.0891089108910891
0.09900990099009901
0.10891089108910891
0.1188118811881188
0.12871287128712872
0.13861386138613863
0.1485148514851485
0.15841584158415842
0.16831683168316833
0.1782178217821782
0.18811881188118812
0.19801980198019803
0.2079207920792079
0.21782178217821782
0.22772277227722773
0.2376237623762376
0.24752475247524752
0.25742574257425743
0.26732673267326734


IndexError: list index out of range

In [ ]:
betting_df.shape

In [ ]:
betting_df['sum_percent'] = (1 / betting_df['team_one_odd']) + (1 / betting_df['team_two_odd'])
betting_df[betting_df['sum_percent'] < 1].sort_values(by=['sum_percent'])